In [144]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import h5py

In [145]:
f = h5py.File('train_catvnoncat.h5', 'r')
t = h5py.File('test_catvnoncat.h5', 'r')

x = f['train_set_x'].value
y = f['train_set_y'].value

x_test = t['test_set_x'].value
y_test = t['test_set_y'].value

c:\python\lib\site-packages\ipykernel_launcher.py:4: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
  after removing the cwd from sys.path.
c:\python\lib\site-packages\ipykernel_launcher.py:5: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
  """
c:\python\lib\site-packages\ipykernel_launcher.py:7: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
  import sys
c:\python\lib\site-packages\ipykernel_launcher.py:8: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
  


In [146]:
x.shape, y.shape, x_test.shape, y_test.shape

((209, 64, 64, 3), (209,), (50, 64, 64, 3), (50,))

In [147]:
x = x.reshape((x.shape[1]*x.shape[2]*x.shape[3],x.shape[0]))
x_test = x_test.reshape((x_test.shape[1]*x_test.shape[2]*x_test.shape[3],x_test.shape[0]))
y = y.reshape((1,y.shape[0]))
y_test = y_test.reshape((1,y_test.shape[0]))

In [148]:
x = x/255
x_test = x_test/255

In [149]:
n_x, n_y = x.shape[0], y.shape[0]

In [150]:
layer_dim = [n_x, 64, 32, n_y]

In [151]:
def sigmoid(Z):
    
    A = 1/(1 + np.exp(-Z))
    
    return A

In [152]:
def relu(Z):

    A = np.maximum(0,Z)

    return A

In [153]:
def d_relu(dA, Z):
    
    dZ = np.array(dA, copy = True)
    dZ[Z <= 0 ] = 0
    
    return dZ

In [154]:
def init_parameters(layer_dimensions):
    
    L = len(layer_dimensions) - 1
    parameters = {}
    
    for l in range(L):
        parameters['W' + str(l + 1)] = np.random.randn(layer_dimensions[l + 1], layer_dimensions[l]) * np.sqrt(2/layer_dimensions[l])
        parameters['b' + str(l + 1)] = np.zeros((layer_dimensions[l + 1], 1))
        
    return parameters    

p = init_parameters(layer_dim)

In [155]:
def forward_propagation(X, parameters):
    
    W1 = parameters['W1']
    W2 = parameters['W2']
    W3 = parameters['W3']
    b1 = parameters['b1']
    b2 = parameters['b2']
    b3 = parameters['b3']
    
    Z1 = np.dot(W1, X) + b1
    A1 = relu(Z1)
    Z2 = np.dot(W2, A1) + b2
    A2 = relu(Z2)
    Z3 = np.dot(W3, A2) + b3
    A3 = sigmoid(Z3)
    
    cache = {'Z1':Z1, 'Z2':Z2, 'Z3':Z3,
            'A1':A1, 'A2':A2, 'A3':A3}
    
    return A3, cache

a2, cac = forward_propagation(x, p)

In [192]:
def compute_cost(Y, A3, parameters, lambd):
    
    m = Y.shape[1]
    W1 = parameters["W1"]
    W2 = parameters["W2"]
    W3 = parameters["W3"]
    
    regularized_term = (1/m * lambd/2) * (np.sum(np.square(W1)) + np.sum(np.square(W2)) + np.sum(np.square(W3)))

    cost = (-1/m) * np.sum((Y * np.log(A3)) + ((1-Y) * np.log(1-A3))) + regularized_term
    
    return cost

compute_cost(y, a2)

In [198]:
def backward_propagation(X, Y, cache, parameters, lambd):
   
    m = Y.shape[1]
    Z1 = cache['Z1']
    Z2 = cache['Z2'] 
    A1 = cache['A1']
    A2 = cache['A2']
    A3 = cache['A3']
    W1 = parameters['W1']
    W2 = parameters['W2']
    W3 = parameters['W3']
       
    
    
    dZ3 = A3 - Y
    dW3 = (1/m) * np.dot(dZ3, A2.T) + (lambd * W3/ m)
    db3 = (1/m) * np.sum(dZ3, axis = 1, keepdims=True)
    
    dA2 = np.dot(W3.T, dZ3)
    dZ2 = d_relu(dA2, Z2)
    dW2 = (1/m) * np.dot(dZ2, A1.T) + (lambd * W2/ m)
    db2 = (1/m) * np.sum(dZ2, axis = 1, keepdims=True) 
    
    dA1 = np.dot(W2.T, dZ2)
    dZ1 = d_relu(dA1, Z1)
    dW1 = (1/m) * np.dot(dZ1, X.T) + (lambd * W1/ m)
    db1 = (1/m) * np.sum(dZ1, axis = 1, keepdims=True)    
    
    grads = {'dW1' : dW1, 'dW2' : dW2, 'dW3' : dW3, 'db1' : db1, 'db2' : db2, 'db3' : db3}
    
    return grads

g = backward_propagation(x, y, cac, p)

In [199]:
def update_parameters(parameters, grads, learning_rate):
    
    L = len(parameters)//2 
    for l in range(L):
        parameters['W' + str(l+1)] -= (learning_rate *  grads['dW' + str(l+1)])
        parameters['b' + str(l+1)] -= (learning_rate *  grads['db' + str(l+1)])
                
            
    return parameters

par = update_parameters(p, g, 0.01)

In [200]:
def model(X, Y, layer_dimensions, learning_rate, iterations, lambd):
    
    np.random.seed(42)
    costs = []
    parameters = init_parameters(layer_dimensions)
    
    for i in range(iterations):
        A3, cache = forward_propagation(X, parameters)
        cost = compute_cost(Y, A3, parameters, lambd)
        grads = backward_propagation(X, Y, cache, parameters, lambd)
        parameters = update_parameters(parameters, grads, learning_rate)
        
        if i % 100 == 0:
            print(f'Iteration {i}: {cost}')
         
        costs.append(cost)
         
    plt.plot(costs)
    
    return parameters

In [225]:
para = model(x, y, layer_dim, 0.008, 10000, lambd = 0.8)

Iteration 0: 1.1179847618117582
Iteration 100: 0.9366731944871696
Iteration 200: 0.8915670055109911
Iteration 300: 0.8809373755999206
Iteration 400: 0.8328690975096522
Iteration 500: 0.8021528679876858
Iteration 600: 0.7796173710966312
Iteration 700: 0.7685420553467845
Iteration 800: 0.7550347307379821
Iteration 900: 0.7488321187085634
Iteration 1000: 0.7054904634917261
Iteration 1100: 0.6760813868745454
Iteration 1200: 0.6520240117974452
Iteration 1300: 0.6419411257874703
Iteration 1400: 0.6098852858588089
Iteration 1500: 0.5808148835798452
Iteration 1600: 0.5723726719889134
Iteration 1700: 0.578666343983866
Iteration 1800: 0.5460852704516462
Iteration 1900: 0.5493022224179541
Iteration 2000: 0.5012588399386237
Iteration 2100: 0.7383288328671185
Iteration 2200: 0.5499391311635364
Iteration 2300: 0.471519090777448
Iteration 2400: 0.467571922519091
Iteration 2500: 0.46188088238790864
Iteration 2600: 0.46678218693434087
Iteration 2700: 0.45894101890048744


KeyboardInterrupt: 

In [218]:
a3, _ = forward_propagation(x, para)
a3_test, _ = forward_propagation(x_test, para)

In [33]:
def predict(A3, Y, parameters):

    m = A3.shape[1]
    n = len(parameters) // 2 
    p = np.zeros((1,m))
    

    for i in range(0, A3.shape[1]):
        if A3[0,i] > 0.5:
            p[0,i] = 1
        else:
            p[0,i] = 0
    
    print("Accuracy: "  + str(np.sum((p == Y)/m)))
        
    return p

In [219]:
preditction_train = predict(a3, y, para)
prediction_test = predict(a3_test, y_test, para)

Accuracy: 0.9999999999999998
Accuracy: 0.3400000000000001


(array([[0.09377708, 0.05263986, 0.01823401, 0.0815813 , 0.05250223,
         0.01703311, 0.06003271, 0.04052315, 0.00877709, 0.04939296,
         0.03404341, 0.01021964, 0.05009573, 0.06226629, 0.02197579,
         0.07256279, 0.05315119, 0.00808184, 0.0809952 , 0.03466802,
         0.00866693, 0.04069036, 0.06924376, 0.01335105, 0.03638862,
         0.10643341, 0.01279572, 0.05128548, 0.17666575, 0.02375618,
         0.05503652, 0.16954792, 0.01670585, 0.03111611, 0.07444345,
         0.02556668, 0.06695237, 0.06980081, 0.02518984, 0.12779151,
         0.10377407, 0.0437681 , 0.21850567, 0.09061706, 0.02178376,
         0.20660939, 0.04674611, 0.01081852, 0.13154232, 0.0787438 ]]),
 array([[1.26297663e-01, 1.02448505e-02, 7.81570640e-01, 1.71206027e-02,
         3.00674276e-02, 1.60988958e-01, 4.27867960e-03, 7.48437848e-01,
         1.64160948e-01, 7.02720074e-04, 1.53691323e-01, 9.19868034e-01,
         8.02606695e-03, 8.80163976e-01, 9.48632948e-01, 5.92181462e-03,
         1.5770